<a href="https://colab.research.google.com/github/keyom-ai/rag/blob/main/retrieval_augment_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This Notebook is created to solve RAG (retrieval augmented generation) use case - in this example we are feeding an external text file stored in a website.


This section of the code is installing required libraries:
- [langchain](https://www.langchain.com/): mainly used for orchestration and chaining of prompts
- [weaviate](https://weaviate.io/): open-source vector database
- openai: no explanation needed :)
- tiktoken: [tokenizer](https://github.com/openai/tiktoken) used by OpenAI cookbook

---



In [ ]:
!pip install langchain openai weaviate-client python-dotenv tiktoken

This is setting up the environment variable OPENAI_API_KEY. Please make sure to replace `sk-abcdefghijklmnopqrstuvwxyz0123456789` with your own OpenAI API key. You can create an API key via their [API key page](https://platform.openai.com/api-keys).

**A note: ** You may need to [add](https://platform.openai.com/account/billing/overview) $10 (minimum) in your API balance if you start getting capacity error or error code 401

---



In [32]:
import os
import openai
OPENAI_API_KEY='sk-abcdefghijklmnopqrstuvwxyz0123456789'

In [33]:
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


This is the code block which where we are retrieving the external data -- in this case it's **State Of The Union** we are going to use.

Once retrieved from the URL, we are writing this as a file and loading it.

In [34]:
import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()


This step is mainly spillitng that file  into chunks so it can be fed to Embeddings model and then it can be stored in the vector database.

In [35]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)


This is where we are telling langchain to use OpenAI Embeddings model and convert our text document into embeddings and store it into vector databse - in our case we are using **weaviate**

There are many other Embeddings model exist, such as [Amazon Titan Embeddings](https://docs.aws.amazon.com/bedrock/latest/userguide/embeddings.html), [Cohere Embeddings](https://docs.cohere.com/docs/multilingual-language-models) and many more via [HuggingFace](https://huggingface.co/models?other=embeddings)

**A note: ** You may need to [add](https://platform.openai.com/account/billing/overview) $10 (minimum) in your API balance if you start getting capacity error or error code 401 *italicized text*

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

#OPENAI_API_KEY='sk-abcdefghijklmnopqrstuvwxyz0123456789'

os.environ["OPENAI_API_KEY"] = "sk-abcdefghijklmnopqrstuvwxyz0123456789"

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)


In this code block we are retrieving the information from the vector database.

In [37]:
retriever = vectorstore.as_retriever()

In this code block we are getting the prompt ready for consumption.

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)


Here is we are using the prompt and adding context via embedding store to reflect current information.

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "What did the president say about Justice Breyer"
rag_chain.invoke(query)

"The president thanked Justice Breyer for his service and acknowledged his dedication to serving the country. The president also mentioned that he nominated Judge Ketanji Brown Jackson as a successor to continue Justice Breyer's legacy of excellence."